In [1]:
!pip install -U pymilvus
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.3/197.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully uninstalled grpcio-1.64.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [1]:
from pymilvus import MilvusClient
from pymilvus import FieldSchema, DataType
from pymilvus import FieldSchema, CollectionSchema

import pandas as pd
import numpy as np
import time
import openai
from openai import OpenAI
import os

In [28]:
INDEX_TYPE = "FLAT"
DIMENSION = 768
METRIC_TYPE = "IP"
NUM_PARTITIONS = 17

class MakeCollections:
  def __init__(self, client, index_type, metric_type, dimension):
    self.client = client
    self.index_type = index_type
    self.metric_type = metric_type
    self.dimension = dimension


  # 스키마 생성
  def create_schema(self):
    fields = [
      FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
      FieldSchema(name="place_name", dtype=DataType.VARCHAR, max_length=100, description="the name of place"),
      FieldSchema(name="category", dtype=DataType.VARCHAR, max_length=30, description="the category of place"),
      FieldSchema(name="area_name", dtype=DataType.VARCHAR, max_length=100, description="the name of administrative district"),
      FieldSchema(name="reservation", dtype=DataType.BOOL, description="the reservation availability of place"),
      FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=500, description="elements of travel sites"),
      FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=self.dimension, description="vector"),
    ]
    schema = CollectionSchema(fields=fields, description="travel sites", partition_key_field="area_name")
    return schema

  # 인덱스 생성
  def create_index(self):
    index_params = self.client.prepare_index_params()

    index_params.add_index(
      field_name="embedding",
      index_type=self.index_type,
      metric_type=self.metric_type
    )

    return index_params

  # 컬렉션 생성
  def create_collection(self, collection_name):
    self.client.create_collection(
      collection_name=collection_name,
      schema=self.create_schema(),
      index_params=self.create_index(),
      num_partitions=NUM_PARTITIONS
    )

    time.sleep(2)

    res = self.client.get_load_state(
      collection_name=collection_name
    )
    print(res)

    return self.client


In [3]:
from google.colab import userdata
url = userdata.get("URL")

# 데이터베이스 연결
client = MilvusClient(url)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 24b887b5db494f36b30069c2ad2f1bca


In [ ]:
# 존재하는 컬렉션 제거
client.drop_collection(
    collection_name="kstartup_travel_sites"
)


client.drop_collection(
    collection_name="nowlocal_travel_sites"
)


In [ ]:
collection = MakeCollections(client, INDEX_TYPE, METRIC_TYPE, DIMENSION)
kstartup_collection = collection.create_collection("kstartup_travel_sites")
nowlocal_collection = collection.create_collection("nowlocal_travel_sites")

In [40]:
# 만들어진 컬렉션 확인
client.list_collections()

['kstartup_travel_sites', 'nowlocal_travel_sites']

In [41]:
# 만든 컬렉션 확인
res = client.describe_collection(
    collection_name="kstartup_travel_sites"
)
res

{'collection_name': 'kstartup_travel_sites',
 'auto_id': False,
 'num_shards': 1,
 'description': 'travel sites',
 'fields': [{'field_id': 100,
   'name': 'id',
   'description': '',
   'type': <DataType.INT64: 5>,
   'params': {},
   'is_primary': True},
  {'field_id': 101,
   'name': 'place_name',
   'description': 'the name of place',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 100}},
  {'field_id': 102,
   'name': 'category',
   'description': 'the category of place',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 30}},
  {'field_id': 103,
   'name': 'area_name',
   'description': 'the name of administrative district',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 100},
   'is_partition_key': True},
  {'field_id': 104,
   'name': 'reservation',
   'description': 'the reservation availability of place',
   'type': <DataType.BOOL: 1>,
   'params': {}},
  {'field_id': 105,
   'name': 'text',
   'description': 'elements of travel site

In [15]:
#--------데이터 입력하기---------

#openai api 쓰기 위한 환경변수 설정
EMBEDDINGS_KEY = userdata.get('EMBEDDINGS_KEY')
os.environ["OPENAI_API_KEY"] = EMBEDDINGS_KEY

openAI_api = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [16]:
# 임베딩 하는 함수
def embed_string(string):
    response = openAI_api.embeddings.create(
        input=string,
        model="text-embedding-3-small",
        dimensions=768
    )
    embedding = response.data[0].embedding
    return embedding

In [42]:
# 데이터 가져올 파일명
data_list = {
    # 컬렉션 이름 : 가져올 데이터 파일이름
    #'nowlocal_travel_sites' : 'data_nowlocal_time.xlsx',
    'kstartup_travel_sites' : 'data_kstartup_time.xlsx'
}


# 행정구역 구분
sections = ["서울특별시", "부산광역시", "인천광역시", "대구광역시", "대전광역시",
            "광주광역시", "울산광역시", "세종특별자치시", "경기도", "충청북도",
            "충청남도", "전라남도", "경상북도", "경상남도", "강원특별자치도",
            "전북특별자치도", "제주특별자치도"]



def make_dataset(row):
    id = row['store ID']
    place_name = row['상호명']
    category = row['카테고리']
    area_name = row['주소1'].split()[0]
    reservation = row['예약 가능 여부']

    string_data = f"""장소명: {row['상호명']}
카테고리: {row['카테고리']}
장소 키워드: {row['키워드']}
위치: {row['주소2']}
해시태그: {row['해시태그']}
"""
    vector = embed_string(string_data)

    return {'id': id, 'place_name':place_name, 'category':category, 'area_name':area_name, 'reservation': reservation, 'text':string_data, 'embedding':vector}


def make_document_hashtag(row):
  city = row['주소2'][0:2]
  area_1 = row['주소1'].split()[1]
  area_2 = row['주소1'].split()[1][0:-1]
  category = row['카테고리']

  return f"{city}, {area_1}, {area_2}, {category}, 추천 여행지, 놀만한 곳, 가볼 만한 곳"


for collection_name, file_name in data_list.items():
  # 엑셀 파일 읽어오기
  original_df = pd.read_excel(file_name, engine='openpyxl')

  # 필요없는 열 버리기
  columns_to_drop = ['설명','(네이버)평점', '(카카오맵)평점', '운영정보']
  original_df = original_df.drop(columns=columns_to_drop)

  # 원본 파일 복사
  copy_df = original_df.copy()

  for section in sections:
    section_df = copy_df[copy_df['주소1'].str.contains(section)].copy()

    # 문서 추가 키워드 생성
    section_df["해시태그"] = section_df.apply(make_document_hashtag, axis=1)

    dataset = section_df.apply(make_dataset, axis=1).tolist()

    client.insert(
      collection_name=collection_name,
      data=dataset
    )

In [43]:
collection_list = ['kstartup_travel_sites', 'nowlocal_travel_sites']

# 컬렉션 구체적 정보(인덱스 지정 필드, 삽입 데이터 개수) 확인
for collection_name in collection_list:
  print(f'-- {collection_name} --')
  res = client.list_indexes(collection_name=collection_name)
  print(f'인덱스 지정된 필드 리스트: {res}')

  res = client.query(
      collection_name=collection_name,
      filter="",
      output_fields=["count(*)"]
  )
  print(f'삽입 데이터 개수: {res}')
  print()


-- kstartup_travel_sites --
인덱스 지정된 필드 리스트: ['embedding']
삽입 데이터 개수: data: ["{'count(*)': 244}"] 

-- nowlocal_travel_sites --
인덱스 지정된 필드 리스트: ['embedding']
삽입 데이터 개수: data: ["{'count(*)': 154}"] 



In [44]:
# 인덱스 정보 확인
for collection_name in collection_list:
  res = client.describe_index(
    collection_name=collection_name,
    index_name="embedding"
  )
  print(res)

  '''
  # 6. Drop index
  client.drop_index(
    collection_name=collection_name,
    index_name="embedding"
  )
  '''


{'index_type': 'FLAT', 'metric_type': 'IP', 'field_name': 'embedding', 'index_name': 'embedding', 'total_rows': 0, 'indexed_rows': 0, 'pending_index_rows': 0, 'state': 'Finished'}
{'index_type': 'FLAT', 'metric_type': 'IP', 'field_name': 'embedding', 'index_name': 'embedding', 'total_rows': 0, 'indexed_rows': 0, 'pending_index_rows': 0, 'state': 'Finished'}


In [ ]:
# 필터링 검색 테스트
for collection_name in collection_list:
  res = client.query(
    collection_name=collection_name,
    filter="category == '음식점'",
    output_fields=["text", "reservation"],
    limit=10
  )
  print(res)


In [48]:
client.close()